In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from fpylll import IntegerMatrix, LLL

In [2]:
def fetch_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    data = data.dropna(axis=1, how='all')  # Drop columns with all NaN values
    data = data.dropna(axis=0, how='all')  # Drop rows with all NaN values
    data = data.ffill()  # Fill NaN values with the previous valid observation
    return data

In [3]:
def standardise_risk_matrix(raw_risk_matrix):
    risk_matrix = raw_risk_matrix.copy()
    # Min-max scale Volatility and Amihud Illiquidity to [0, 1]
    for col in ['Volatility', 'Amihud Illiquidity']:
        if col in risk_matrix.columns:
            min_val = risk_matrix[col].min()
            max_val = risk_matrix[col].max()
            risk_matrix[col] = (risk_matrix[col] - min_val) / (max_val - min_val)

    return risk_matrix

In [4]:
def calculate_risk_matrix(data):
    close_data = data['Close'].dropna(axis=1, how='any')
    volume_data = data['Volume'].dropna(axis=1, how='any')
    volume_data = volume_data.replace(0, np.nan).ffill()
    returns = close_data.pct_change().dropna()  # Calculate daily returns
    sp500_returns = returns['^GSPC']  # S&P 500 returns
    betas = returns.corrwith(sp500_returns) # Beta of each stock
    volatilities = returns.std()  # Volatility of each stock
    amihud_illiquidity = ((1e6)*(np.abs(returns)/volume_data).mean())  # Amihud Illiquidity Measure
    left_threshold = returns['^GSPC'].quantile(0.05)
    right_threshold = returns['^GSPC'].quantile(0.95)
    tail_data = returns[(returns['^GSPC'] >= right_threshold) | (returns['^GSPC'] <= left_threshold)]
    tail_corr = tail_data.corr()
    tail_risk = tail_corr['^GSPC']
    raw_risk_matrix = pd.DataFrame({
        'Beta': betas,
        'Volatility': volatilities,
        'Amihud Illiquidity': amihud_illiquidity,
        'Tail Risk': tail_risk
    })
    # standardize the risk matrix using min-max scaling
    risk_matrix = standardise_risk_matrix(raw_risk_matrix)
    # drop GSPC row
    risk_matrix = risk_matrix.drop(index='^GSPC', errors='ignore')
    # shuffle order of rows
    risk_matrix = risk_matrix.sample(frac=1, random_state=42)

    return risk_matrix

In [5]:
def find_stock_combinations_LLL(risk_matrix):
    # Scale to integers if needed (LLL requires integer matrix)
    scale = 1e6
    basis = np.round(risk_matrix.values * scale).astype(int)
    n_rows, n_cols = basis.shape
    np.random.seed(42)  # For reproducibility
    pad = np.random.randint(0, 2, size=(n_rows, n_rows - n_cols))
    basis = np.hstack([basis, pad])
    R = IntegerMatrix.from_matrix(basis)
    U = IntegerMatrix.identity(R.nrows)
    LLL.reduction(R, U)
    # short_vec = np.array(R[0])
    # coeffs = np.array(U[0])
    return U

In [6]:
# # Scrape S&P 500 tickers from Wikipedia
# url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# table = pd.read_html(url)
# sp500 = table[0]
# tickers = sp500['Symbol'].tolist()

# # convert tickers to yfinance format
# tickers = [ticker.replace('.', '-') for ticker in tickers]
# tickers = ['^GSPC'] + tickers  # Add S&P 500 index ticker

# # Fetch stock data for the S&P 500 companies
# start_date = '2020-01-01'
# end_date = '2023-01-01'
# data = fetch_stock_data(tickers, start_date, end_date)

data = pd.read_pickle('sp500_data.pkl')

In [7]:
risk_matrix = calculate_risk_matrix(data)

In [8]:
U = find_stock_combinations_LLL(risk_matrix)
z = np.zeros((risk_matrix.shape[0], risk_matrix.shape[0]), dtype=int)
_ = U.to_matrix(z)
# create a dictionary to map each row index to the corresponding stock combinations and the quantity
stock_combinations_lll = {
    i:  {
        risk_matrix.index[j]: z[i, j] for j in range(len(z[i])) if z[i, j] != 0
    }
    for i in range(z.shape[0])
}

In [9]:
def find_stock_combinations_cvx(risk_matrix, max_stocks=50, min_stocks=1):
    # Create variables
    n_stocks = len(risk_matrix)
    positions = cp.Variable(n_stocks, integer=True)
    y = cp.Variable(n_stocks, boolean=True)  # Indicator: 1 if stock is included, 0 otherwise

    M = 100000
    
    # Create risk matrix as numpy array
    R = risk_matrix.values
    
    # Objective: Minimize Euclidean norm of risk vector
    objective = cp.Minimize(cp.norm(R.T @ positions, 2))
    
    # Constraints
    constraints = [
        cp.sum(cp.abs(positions) != 0) <= max_stocks,  # Max stocks constraint
        cp.sum(cp.abs(positions) != 0) >= min_stocks,  # Min stocks constraint
    ]
    
    # Solve problem
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS_BB)
    
    if positions.value is not None:
        # Create portfolio dictionary
        portfolio = {}
        for i, stock in enumerate(risk_matrix.index):
            if abs(positions.value[i]) > 0.5:  # Threshold to handle numerical issues
                portfolio[stock] = int(round(positions.value[i]))
        
        # Calculate risk of the portfolio
        portfolio_risk = np.linalg.norm(
            R.T @ np.array([portfolio.get(stock, 0) for stock in risk_matrix.index])
        )
        
        return {
            'portfolio': portfolio,
            'num_stocks': len(portfolio),
            'portfolio_risk': portfolio_risk
        }
    else:
        print("Optimization failed.")
        return None

In [52]:
n_stocks = len(risk_matrix)
positions = cp.Variable(n_stocks)
y_pos = cp.Variable(n_stocks, boolean=True)
y_neg = cp.Variable(n_stocks, boolean=True)

M = 100
min_position = 1

# Create risk matrix as numpy array
R = risk_matrix.values

# Objective: Minimize Euclidean norm of risk vector
objective = cp.Minimize(cp.norm(R.T @ positions, 2))

# Constraints
constraints = [
    # cp.sum(positions) >= 1,   # Min stocks constraint
    cp.sum(y_pos) >= 1,
    cp.sum(y_neg) >= 1,
]

for i in range(n_stocks):
    constraints.append(y_pos[i] + y_neg[i] <= 1)  # Only one of pos/neg per stock

    constraints.append(positions[i] >= min_position * y_pos[i] - M * (1 - y_pos[i]))
    constraints.append(positions[i] <= M * y_pos[i])

    constraints.append(positions[i] <= -min_position * y_neg[i] + M * (1 - y_neg[i]))
    constraints.append(positions[i] >= -M * y_neg[i])

# Solve problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.ECOS_BB)

/home/linuxiguana/miniforge3/envs/python-dev/lib/python3.13/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


np.float64(0.002725165760158103)

In [ ]:
p = np.round(positions.value)

In [57]:
max(p), min(p), np.sum(p != 0)

(np.float64(4.0), np.float64(-6.0), np.int64(3))

In [60]:
sum(np.round(y_pos.value)), sum(np.round(y_neg.value))

(np.float64(2.0), np.float64(1.0))